In [2]:
# import lib
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt

np.random.seed(78)
tf.set_random_seed(678)

In [31]:
# declare
num_epoch    = 10
ground_truth = np.random.uniform(0,1,10)
expected     = np.zeros(10)
pull_count   = np.zeros(10) + 1e-10
print(ground_truth)
print(expected)
print(pull_count)

[0.19452121 0.91838531 0.68906848 0.24437955 0.63711696 0.27985914
 0.94169872 0.05066309 0.88659284 0.49082195]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.e-10 1.e-10 1.e-10 1.e-10 1.e-10 1.e-10 1.e-10 1.e-10 1.e-10 1.e-10]


In [29]:
# train 
for iter in range(num_epoch):
    current_action = np.random.randint(10)
    pull_count[current_action] = pull_count[current_action] + 1
    if np.random.uniform(0,1) < ground_truth[current_action]: current_reward = 1
    else: current_reward = 0
    
    expected[current_action] = expected[current_action] + (1/pull_count[current_action])

0
0.9109796506321873
0
------
7
0.2414776262567656
0
------
3
0.5519274902548998
0
------
9
0.24400811383276544
0
------
7
0.2414776262567656
0
------
5
0.291016382189513
0
------
1
0.3443528034183553
0
------
8
0.3206369844038238
0
------
7
0.2414776262567656
1
------
0
0.9109796506321873
1
------


# Reference 
1. Solving the Multi-Armed Bandit Problem – Towards Data Science. (2017). Towards Data Science. Retrieved 24 December 2018, from https://towardsdatascience.com/solving-the-multi-armed-bandit-problem-b72de40db97c
2. 